In [20]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, transpile

from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit_ibm_runtime import SamplerV2 as Sampler

service = QiskitRuntimeService()
backend = service.backend("ibm_rensselaer", instance="rpi-rensselaer/classes/itws-4940-callab")


q = QuantumRegister(12, 'q')
c = ClassicalRegister(1, 'c')

circuit = QuantumCircuit(q,c)

### Addressing ###

#q[0],q[1] -> Memory cell
#  0    0  -> 00
#  0    1  -> 01
#  1    0  -> 10
#  1    1  -> 11

#circuit.x(q[0])
#circuit.x(q[1])

### Routing nodes ####
circuit.cx(q[0],q[2])
circuit.x(q[3])
circuit.cx(q[2],q[3])

circuit.ccx(q[1],q[2],q[4])
circuit.cx(q[4],q[2])

circuit.ccx(q[1],q[3],q[5])
circuit.cx(q[5],q[3])

circuit.x(q[11]) #Write mode (read mode if commented)

### Writing to memory cell ###
circuit.ccx(q[11],q[4],q[9])
circuit.ccx(q[11],q[5],q[8])
circuit.ccx(q[11],q[2],q[7])
circuit.ccx(q[11],q[3],q[6])

circuit.barrier(q)

### Reading memory cell ###
circuit.ccx(q[4],q[9],q[10])
circuit.ccx(q[5],q[8],q[10])
circuit.ccx(q[2],q[7],q[10])
circuit.ccx(q[3],q[6],q[10])

circuit.barrier(q)
circuit.measure(q[10],c[0]) # Measuring readout qubit 

circuit.draw(output='mpl', filename='QRAM.png')
 

compiled_circuit = transpile(circuit, backend)
sampler = Sampler(backend)
sampler.options.default_shots = 100
job = sampler.run([compiled_circuit])           


result = job.result() 

pub_result = job.result()[0]

/tmp/ipykernel_1578/763014813.py:8: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, ibm_cloud will be the only valid channel. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService()


In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
import numpy as np
from scipy.linalg import hadamard

# ── Qiskit-Experiments imports ───────────────────────────────────────────────
from qiskit_experiments.framework import ExperimentData, ParallelExperiment
from qiskit_experiments.library.randomized_benchmarking import StandardRB

# ── 0) Set up IBM runtime & backend ──────────────────────────────────────────
service = QiskitRuntimeService()
backend = service.backend(
    "ibm_rensselaer",
    instance="rpi-rensselaer/classes/itws-4940-callab"
)

# ── 1) Characterize the device (one-time) ────────────────────────────────────
# We run multi-qubit Clifford RB of lengths [1,5,10,20] on all 12 qubits,
# drawing 5 random sequences per length.
char_exp = StandardRB(
    qubits=list(range(12)),
    lengths=[1, 5, 10, 20],
    num_samples=5
)
# Wrap in a ParallelExperiment just to use .run(block_for_results=True)
pexp = ParallelExperiment([char_exp])
exp_data: ExperimentData = pexp.run(
    service=service,
    backend=backend,
    options={"shots": 2000}
).block_for_results()

# Extract the fitted SPAM eigenvalues A_i and gate‐error eigenvalues λ_i
fit_data = pexp.analysis.run(exp_data).data()
A_eigs      = np.array(fit_data["A"])         # length 2^12
lambda_eigs = np.array(fit_data["lambdas"])   # length 2^12

# Build the Pauli-channel matrices:
W  = hadamard(2**12) / np.sqrt(2**12)         # Walsh–Hadamard
N0 = W.T @ np.diag(A_eigs)      @ W          # SPAM matrix
M0 = W.T @ np.diag(lambda_eigs) @ W          # single‐layer gate error

# ── 2) Build your QRAM circuit ──────────────────────────────────────────────
q = QuantumRegister(12, "q")
c = ClassicalRegister(1, "c")
qc = QuantumCircuit(q, c)

# … your addressing/routing/writing/reading gates exactly as before …
qc.cx(q[0], q[2])
qc.x(q[3])
qc.cx(q[2], q[3])
qc.ccx(q[1], q[2], q[4])
qc.cx(q[4], q[2])
qc.ccx(q[1], q[3], q[5])
qc.cx(q[5], q[3])
qc.x(q[11])
qc.ccx(q[11], q[4], q[9])
qc.ccx(q[11], q[5], q[8])
qc.ccx(q[11], q[2], q[7])
qc.ccx(q[11], q[3], q[6])
qc.barrier(q)
qc.ccx(q[4], q[9], q[10])
qc.ccx(q[5], q[8], q[10])
qc.ccx(q[2], q[7], q[10])
qc.ccx(q[3], q[6], q[10])
qc.barrier(q)
qc.measure(q[10], c[0])

# ── 3) Transpile then compute Qₘ⁻¹ ──────────────────────────────────────────
compiled = transpile(qc, backend)
m = compiled.depth()  # circuit “depth” → number of layers
Qm = N0 @ np.linalg.matrix_power(M0, m)
Qm_inv = np.linalg.inv(Qm)

# ── 4) Run with SamplerV2 ──────────────────────────────────────────────────
sampler = Sampler(backend=backend, service=service)
sampler.options.default_shots = 100
job = sampler.run([compiled])
res = job.result()

# SamplerV2 returns a list of quasi‐probability distributions:
quasi_dist = res.quasi_dists[0]  # this is a dict: bitstring→probability

# Convert to a probability vector q_noisy
dim = 2**12
q_noisy = np.zeros(dim)
for bs, p in quasi_dist.binary_probabilities().items():
    idx = int(bs[::-1], 2)
    q_noisy[idx] = p

# Apply mitigation: q_mit = Qₘ⁻¹ ⋅ q_noisy
q_mit = Qm_inv.dot(q_noisy)

# Turn back into counts‐like dict for comparison
shots = sampler.options.default_shots
mit_counts = {
    bs[::-1]: int(q_mit[int(bs,2)] * shots)
    for bs in quasi_dist.binary_probabilities().keys()
}

print("Raw counts:    ", res.quasi_dists[0].binary_probabilities())
print("Mitigated freq:", mit_counts)

/tmp/ipykernel_5667/428763639.py:11: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, ibm_cloud will be the only valid channel. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService()


TypeError: StandardRB.__init__() got an unexpected keyword argument 'qubits'

In [21]:
counts = pub_result.data.c.get_counts()
print(f">>> Hardware counts: {counts}")

print(circuit)
print(counts)

>>> Hardware counts: {'0': 59, '1': 41}
                                                         ░                     »
 q_0: ──■────────────────────────────────────────────────░─────────────────────»
        │                                                ░                     »
 q_1: ──┼─────────■─────────■────────────────────────────░─────────────────────»
      ┌─┴─┐       │  ┌───┐  │                            ░                     »
 q_2: ┤ X ├──■────■──┤ X ├──┼───────────────────■────────░─────────────■───────»
      ├───┤┌─┴─┐  │  └─┬─┘  │  ┌───┐            │        ░             │       »
 q_3: ┤ X ├┤ X ├──┼────┼────■──┤ X ├────────────┼────■───░─────────────┼────■──»
      └───┘└───┘┌─┴─┐  │    │  └─┬─┘            │    │   ░             │    │  »
 q_4: ──────────┤ X ├──■────┼────┼────■─────────┼────┼───░───■─────────┼────┼──»
                └───┘     ┌─┴─┐  │    │         │    │   ░   │         │    │  »
 q_5: ────────────────────┤ X ├──■────┼────■────┼────┼───░───┼────■──

test on simulator

In [16]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import Aer


q = QuantumRegister(12, 'q')
c = ClassicalRegister(1, 'c')

circuit = QuantumCircuit(q,c)

### Addressing ###

#q[0],q[1] -> Memory cell
#  0    0  -> 00
#  0    1  -> 01
#  1    0  -> 10
#  1    1  -> 11

#circuit.x(q[0])
#circuit.x(q[1])

### Routing nodes ####
circuit.cx(q[0],q[2])
circuit.x(q[3])
circuit.cx(q[2],q[3])

circuit.ccx(q[1],q[2],q[4])
circuit.cx(q[4],q[2])

circuit.ccx(q[1],q[3],q[5])
circuit.cx(q[5],q[3])

circuit.x(q[11]) #Write mode (read mode if commented)

### Writing to memory cell ###
circuit.ccx(q[11],q[4],q[9])
circuit.ccx(q[11],q[5],q[8])
circuit.ccx(q[11],q[2],q[7])
circuit.ccx(q[11],q[3],q[6])

circuit.barrier(q)

### Reading memory cell ###
circuit.ccx(q[4],q[9],q[10])
circuit.ccx(q[5],q[8],q[10])
circuit.ccx(q[2],q[7],q[10])
circuit.ccx(q[3],q[6],q[10])

circuit.barrier(q)
circuit.measure(q[10],c[0]) # Measuring readout qubit 

circuit.draw(output='mpl', filename='QRAM.png')
 
shots = 100
backend = Aer.get_backend('qasm_simulator')
# job = execute(qc, backend, shots=shots)
new_circuit = transpile(circuit, backend)
job = backend.run(new_circuit, shots=shots)
counts = job.result().get_counts(new_circuit)
print(counts)


{'1': 100}


less noise?

In [17]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler

# ─── 1) Initialize IBM Runtime service & backend ──────────────────────────────
service = QiskitRuntimeService()
backend = service.backend(
    "ibm_rensselaer",
    instance="rpi-rensselaer/classes/itws-4940-callab"
)

# ─── 2) Build your 12-qubit QRAM circuit ────────────────────────────────────
q = QuantumRegister(12, 'q')
c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(q, c)

### Addressing ###
# q[0],q[1] -> memory cell (00,01,10,11)
# (uncomment to test different inputs:)
# circuit.x(q[0])
# circuit.x(q[1])

### Routing nodes ###
circuit.cx(q[0], q[2])
circuit.x(q[3])
circuit.cx(q[2], q[3])

circuit.ccx(q[1], q[2], q[4])
circuit.cx(q[4], q[2])

circuit.ccx(q[1], q[3], q[5])
circuit.cx(q[5], q[3])

# write/read mode toggle
circuit.x(q[11])  # comment this out to switch to “read” mode

### Writing to memory ###
circuit.ccx(q[11], q[4], q[9])
circuit.ccx(q[11], q[5], q[8])
circuit.ccx(q[11], q[2], q[7])
circuit.ccx(q[11], q[3], q[6])

circuit.barrier(q)

### Reading from memory ###
circuit.ccx(q[4], q[9], q[10])
circuit.ccx(q[5], q[8], q[10])
circuit.ccx(q[2], q[7], q[10])
circuit.ccx(q[3], q[6], q[10])

circuit.barrier(q)
circuit.measure(q[10], c[0])

# Optional: save a diagram
# circuit.draw(output='mpl', filename='QRAM.png')


# ─── 3) Transpile with SABRE + high opt level ─────────────────────────────────
compiled = transpile(
    circuit,
    backend,
    optimization_level=3,
    layout_method="sabre",
    routing_method="sabre"
)

# ─── 4) Configure the Sampler with noise-suppression options ─────────────────
sampler = Sampler(backend)
sampler.options.default_shots = 100

# Pauli-twirling on multi-qubit gates
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"
sampler.options.twirling.strategy = "active"

# Dynamical decoupling for idle qubits
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"

# ─── 5) Run and fetch counts ─────────────────────────────────────────────────
job = sampler.run([compiled])
result = job.result()
counts = result[0].data.c.get_counts()

print("Noisy-suppressed hardware counts:", counts)


/tmp/ipykernel_1578/653941361.py:5: DeprecationWarning: The "ibm_quantum" channel option is deprecated and will be sunset on 1 July. After this date, ibm_cloud will be the only valid channel. For information on migrating to the new IBM Quantum Platform on the "ibm_cloud" channel, review the migration guide https://quantum.cloud.ibm.com/docs/migration-guides/classic-iqp-to-cloud-iqp .
  service = QiskitRuntimeService()
/mnt/c/users/maryc/onedrive/desktop/Quantum Research 2025/.venv/lib/python3.10/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:875: UserWarning: IBM Quantum Platform is moving, and this version will be sunset on July 1. To get started on the new platform, read the migration guide at https://docs.quantum.ibm.com/migration-guides/classic-iqp-to-cloud-iqp
  warnings.warn(warning_message)


Noisy-suppressed hardware counts: {'0': 56, '1': 44}


# BUILD SIMPLICES